# core

> Implements the contextpack topics

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from contextkit import *
from fastcore.utils import *
from typing import Callable

import httpx

Define a Topic and SubTopic (ST)

In [ ]:
#| export

class Topic:
    def _children(self):
        return sorted(list(set(c for c in dir(self) if not c.startswith('_')) - {'get','getter','docs','url'}))
    def __iter__(self): 
        yield from (getattr(self, name) for name in self._children() 
                    if isinstance(getattr(self, name),ST))
    def __repr__(self):
        s = ''
        if self.__doc__: s += f"doc: {self.__doc__}\n"
        if (cs := self._children()): s += f"sub-topics: {cs}\n"            
        return s


In [ ]:
#| export

class ST:
    def __init__(self,url,getter:Callable=read_link):
        self.url = url
        self.getter = getter
    def get(self):
        return self.getter(self.url)
    def __repr__(self):
        s = ''
        if self.url: s += f"URL: {self.url}\n"
        return s


We define a topic via a `Topic` subclass, which defines attributes which are `ST`s: 

In [ ]:
#| export
def xget(url): return httpx.get(url).text

In [ ]:
#| export

class CTX_Fastlite_Sqlutils(Topic):
    "LLM contexts on fastlite-sqlutils"
    def __init__(self):
        self.fastlite_index    = ST('https://answerdotai.github.io/fastlite/index.html.md', xget)
        self.fastlite_core     = ST('https://answerdotai.github.io/fastlite/core.html.md', xget)
        self.sqlite_utils_docs = ST('https://sqlite-utils.datasette.io/en/stable/python-api.html',
                                    lambda url:read_url(url,selector='#sqlite-utils-python-library'))


In [ ]:
ctx_fastlite_sqlutils = CTX_Fastlite_Sqlutils()

In [ ]:
ctx_fastlite_sqlutils

doc: LLM contexts on fastlite-sqlutils
sub-topics: ['fastlite_core', 'fastlite_index', 'sqlite_utils_docs']

In [ ]:
ctx_fastlite_sqlutils.fastlite_core

URL: https://answerdotai.github.io/fastlite/core.html.md

Iterating a topic gives you its subtopics:

In [ ]:
type(list(ctx_fastlite_sqlutils)[0])

__main__.ST

In [ ]:
#| eval: false

all_ctxs = [subtopic.get() for subtopic in ctx_fastlite_sqlutils]
len(all_ctxs)

3

In [ ]:
#| eval: false

one_big_ctx = "\n".join([subtopic.get() for subtopic in ctx_fastlite_sqlutils])
len(one_big_ctx)

98408

In [ ]:
#| export
class CTX_Claudette(Topic):
    def __init__(self):
        self.async_docs    = ST('https://claudette.answer.ai/async.html.md', xget)
        self.core_docs     = ST('https://claudette.answer.ai/core.html.md', xget)
        self.toolloop_docs = ST('https://claudette.answer.ai/toolloop.html.md', xget)



In [ ]:
#| export
class CTX_Docker(Topic):
    def __init__(self):
        self.compose_file       = ST('https://docs.docker.com/reference/compose-file/')
        self.docker_cli         = ST('https://docs.docker.com/reference/cli/docker/')
        self.docker_compose_cli = ST('https://docs.docker.com/reference/cli/docker/compose/')
        self.dockerd            = ST('https://docs.docker.com/reference/cli/dockerd/')
        
        # page-based URLs
        url = 'https://docker-py.readthedocs.io/en/stable/{}.html'
        page_names = ['client','configs','containers','images','networks','nodes',
                 'plugins','secrets','services','swarm','volumes','api','tls',]
        pageurls = [(f"pg_{p}",url.format(p)) for p in page_names]
        for name,url in pageurls: setattr(self,name,ST(url))
        

In [ ]:
#| export
class CTX_FastHtml(Topic):
    "LLM-friendly documentation for FastHtml"
    def __init__(self):
        self.fasthtml_llms_ctx = ST('https://docs.fastht.ml/llms-ctx.txt', xget)
        self.hypermedia_summary = ST('https://gist.github.com/jph00/4ad7d35ad79013aded41b5ba535a12a3',read_gist)


In [ ]:
#| export
class CTX_NBClassicServer(Topic):
    def __init__(self):
        self.nbclassic_server = ST('https://nbclassic.readthedocs.io/en/latest/nbclassic.html',
                                   lambda url:read_url(url,self='bd-article'))


In [ ]:
#| export
class CTX_Fastcore(Topic):
    def __init__(self):
        self.fc_llms_ctx   = ST('https://fastcore.fast.ai/llms-ctx.txt', xget)

We take care to name all topics with a `ctx_` prefix in order to support autocomplete.

In [ ]:
#| export
ctx_docker = CTX_Docker()
ctx_fastcore = CTX_Fastcore()
ctx_nbclassic_server = CTX_NBClassicServer()
ctx_fasthtml = CTX_FastHtml()
ctx_claudette = CTX_Claudette()
ctx_fastlite_sqlutils = CTX_Fastlite_Sqlutils()


### Design considerations:

0. Should be easy for anyone to contribute new material, by adding to a python module.

1. Want to enable the user to be able to browse-via-autocomplete over canned contexts by name.

2. Want this autocomplete to work immediately, without the user needing to do anything besides importing the library.

3. Want "lazy fetching" (i.e. evaluation), so merely importing the library does not trigger dozens of network fetches.

4. nice-to-have. a facility for information _about_ the context besides just the name of it. Ideally interoperating with the normal docstring mechanism?

### Findings:

1. Existing solution uses function application to defer loading, but that also blocks providing immediate autocomplete over subtopics.

2. `AttrDict` out-of-the-box produces too many completions for a great UX but maybe goodenough

3. 

### Tentative design

- top-level module namespace defines variables `ctx_fastcore`, `ctx_docker`, etc.., which provide context on major topics like fastcore, docker, etc..
- attributes on these objects will autocomplete over sub-topics within each topic
- each subtopic is represented by an object, which defines `.get()` which will return the context information.
- A repr

- "anything returned by .get() can be fed to an llm"
- ". TAB always lists only what's relevant" (no junk in the autocomplete)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()